__качаем объявления о продаже земельных участков с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/zemelnye_uchastki/prodam/izhs'
    
raw_data_path = 'data/raw/land/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-11-04 17:11:41 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-04 17:11:41 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-04 17:11:44 | AdsListParser: downloader init
[INFO    ] 2022-11-04 17:11:44 | AvitoParser: init
[INFO    ] 2022-11-04 17:11:49 | AvitoParser: 22 pages for read
[INFO    ] 2022-11-04 17:11:49 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-04 17:11:51 | AdsListParser: read page 1
[INFO    ] 2022-11-04 17:11:53 | AdsListParser: read page 2
[INFO    ] 2022-11-04 17:11:55 | AdsListParser: read page 3
[INFO    ] 2022-11-04 17:12:02 | AdsListParser: read page 4
[INFO    ] 2022-11-04 17:12:04 | AdsListParser: read page 5
[INFO    ] 2022-11-04 17:12:05 | AdsListParser: read page 6
[INFO    ] 2022-11-04 17:12:07 | AdsListParser: read page 7
[INFO    ] 2022-11-04 17:12:10 | AdsListParser: read page 8
[INFO    ] 2022-11-04 17:12:13 | AdsListParser: read page 9
[INFO    ] 2022-11-04 17:12:29 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

1022
975


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
849,2437041928,Объявление «Участок 5 сот. (ИЖС)» с фотографией,1800000,,садоводческое товарищество Мекензиевы Горы,"91:04:004003:998.\n\nКомфортно для круглогодичного проживания. \nЭлектричество, интернет, вода городская. \n\nРассмотрю продажу с подрядом по вашему( нашему) проекту. \n\nРассмотрю обмен на автомобиль, стройматериалы, спецтехнику.\n\nСобственник.",19,2022-11-04 17:12:55.140190,sevastopol
316,2497067251,Объявление «Участок 11 сот. (ИЖС)» с фотографией,3500000,,район Северная Сторона,"Продаётся земельный участок (возможно межевание и продажа половины участка) на возвышенности, с шикарным видом на море, расположен фасадом к морю, ровный без перепадов. До пляжа Учкуевка 700 метров. На участке имеется котлован 12х12, под фундамент жилого д",7,2022-11-04 17:12:55.140190,sevastopol
